In [ ]:
#!pip install tensorflow

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Activation, Dense
from tensorflow.keras.optimizers import Adam

2024-10-19 15:02:35.766542: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 15:02:35.782151: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1729321355.793883    8726 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1729321355.797083    8726 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 15:02:35.808878: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# 載入 MNIST 資料庫的訓練資料，並自動分為『訓練組』及『測試組』
(X_train, y_train), (X_test, y_test_org) = mnist.load_data()

# 將 training 的 input 資料轉為3維，並 normalize 把顏色控制在 0 ~ 1 之間
X_train = X_train.reshape(-1, 28, 28) / 255.0
X_test = X_test.reshape(-1, 28, 28) / 255.0
print(X_train.shape, X_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [3]:
# 建立簡單的線性執行的模型
model = Sequential()
# 加 RNN 隱藏層(hidden layer)
# 必須是 3 dimension
model.add(SimpleRNN(units=256, input_shape=(28, 28)))
# 加 output 層
model.add(Dense(units=10, activation="softmax"))

I0000 00:00:1729321380.055094    8726 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
/home/chesterxalan/github/Classwork/Classwork-PythonDLApplicationDevelopment/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
# 編譯: 選擇損失函數、優化方法及成效衡量方式
LR = 0.001  # Learning Rate
adam = Adam(LR)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 256)            │        72,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,530 (295.04 KB)

 Trainable params: 75,530 (295.04 KB)

 Non-trainable params: 0 (0.00 B)

None


In [5]:
# 將 training 的 label 進行 one-hot encoding，例如數字 7 經過 One-hot encoding 轉換後是 0000001000，即第7個值為 1
y_TrainOneHot = to_categorical(y_train)
y_TestOneHot = to_categorical(y_test_org)

# 將 training 的 input 資料轉為2維
X_train_2D = X_train.reshape(60000, 28, 28)
X_test_2D = X_test.reshape(10000, 28, 28)

x_Train_norm = X_train_2D / 255
x_Test_norm = X_test_2D / 255

# 進行訓練, 訓練過程會存在 train_history 變數中
train_history = model.fit(
    x=x_Train_norm,
    y=y_TrainOneHot,
    validation_split=0.2,
    batch_size=800,
    epochs=10,
    verbose=2,
)

Epoch 1/10


I0000 00:00:1729321413.893263    8977 service.cc:148] XLA service 0x7f72f4016eb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729321413.893309    8977 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-10-19 15:03:33.929447: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1729321414.017457    8977 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-10-19 15:03:35.614379: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_473', 284 bytes spill stores, 404 bytes spill loads

2024-10-19 15:03:35.639896: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_473

60/60 - 4s - 70ms/step - accuracy: 0.2492 - loss: 2.0851 - val_accuracy: 0.3973 - val_loss: 1.6941
Epoch 2/10
60/60 - 0s - 6ms/step - accuracy: 0.4878 - loss: 1.5208 - val_accuracy: 0.6081 - val_loss: 1.2676
Epoch 3/10
60/60 - 0s - 6ms/step - accuracy: 0.5238 - loss: 1.4524 - val_accuracy: 0.6177 - val_loss: 1.2374
Epoch 4/10
60/60 - 0s - 6ms/step - accuracy: 0.6203 - loss: 1.1962 - val_accuracy: 0.6815 - val_loss: 1.0158
Epoch 5/10
60/60 - 0s - 6ms/step - accuracy: 0.5996 - loss: 1.2418 - val_accuracy: 0.2974 - val_loss: 1.8950
Epoch 6/10
60/60 - 0s - 6ms/step - accuracy: 0.5686 - loss: 1.3054 - val_accuracy: 0.6998 - val_loss: 0.9934
Epoch 7/10
60/60 - 0s - 6ms/step - accuracy: 0.6947 - loss: 0.9653 - val_accuracy: 0.7461 - val_loss: 0.8481
Epoch 8/10
60/60 - 0s - 7ms/step - accuracy: 0.7500 - loss: 0.8068 - val_accuracy: 0.8090 - val_loss: 0.6655
Epoch 9/10
60/60 - 0s - 6ms/step - accuracy: 0.7786 - loss: 0.7092 - val_accuracy: 0.8195 - val_loss: 0.5958
Epoch 10/10
60/60 - 0s - 6ms/

In [6]:
# 顯示訓練成果(分數)
loss, accuracy = model.evaluate(x_Test_norm, y_TestOneHot)
print(f"test loss: {loss}  test accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8029 - loss: 0.6391
test loss: 0.5802632570266724  test accuracy: 0.8238000273704529


In [7]:
# 預測(prediction)
predictions = np.argmax(model.predict(x_Test_norm[0:20]), axis=-1)
# get prediction result
print("actual :", y_test_org[0:20])
print("predict:", predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
actual : [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
predict: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 5 4]
